In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import torch.utils.data
from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import matplotlib.pyplot as plt

In [2]:
project_root = '/home/ubuntu/disk1/pytorch-GAN-CGAN/'
os.chdir(project_root)

no_cuda = False
cuda_available = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda_available else "cpu")
BATCH_SIZE = 64
EPOCH = 5000
SEED = 8

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda_available else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./MNIST_data', train=False, transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

In [3]:
# Model params
g_input_size = 100     # Random noise dimension coming into generator, per output vector
g_hidden_size = 256   # Generator complexity
g_output_size = 784    # size of generated output vector

d_input_size = 784   # Minibatch size - cardinality of distributions
d_hidden_size = 256   # Discriminator complexity
d_output_size = 1    # Single dimension for 'real' vs. 'fake'


d_learning_rate = 2e-4  # 2e-4
g_learning_rate = 2e-4
optim_betas = (0.9, 0.999)

print_interval = 200

d_steps = 1  # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
g_steps = 1

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(g_input_size, g_hidden_size),
            nn.LeakyReLU(0.2),
            nn.Linear(g_hidden_size, 512),
            nn.Linear(512, g_output_size),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = x.view(x.size(0), g_input_size)
        out = self.model(x)
        return out

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(d_input_size, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, d_hidden_size),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(d_hidden_size, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out = self.model(x.view(x.size(0), d_input_size))
        out = out.view(out.size(0), -1)
        return out

In [6]:

D  = Discriminator().to(device)
G = Generator().to(device)
print(D)
print(G)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.3)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Dropout(p=0.3)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (7): Sigmoid()
  )
)
Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=784, bias=True)
    (4): Tanh()
  )
)


In [7]:
criterion = nn.BCELoss()
lr = 0.0002
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

In [8]:
batch_idx, (data, label) =enumerate(train_loader).__next__()
print(data.shape, label.shape)
real_data = data.to(device)
d_real_decision = D(Variable(real_data))
print(d_real_decision.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1])


In [9]:
D_losses = []
G_losses = []

def train(epoch):
    D.train()
    G.train()
    
    D_losses.clear()
    G_losses.clear()
    for batch_idx, (data, label) in enumerate(train_loader):
        real_data = data.to(device) 
        D.zero_grad()
        
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            #  1A: Train D on real
            
            d_real_decision = D(Variable(real_data))
#             print(d_real_decision.shape, flush=True, end='')
#             if d_real_decision.shape[0] != BATCH_SIZE:
#                 print(d_real_decision.shape)
            d_real_error = criterion(d_real_decision, Variable(torch.ones(d_real_decision.shape[0],1)))  # ones = true
            d_real_error.backward() # compute/store gradients, but don't change params

            #  1B: Train D on fake
            d_gen_input = Variable(torch.rand(BATCH_SIZE, g_input_size))
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            
            d_fake_decision = D(d_fake_data)
            d_fake_error = criterion(d_fake_decision, Variable(torch.zeros(BATCH_SIZE, 1)))  # zeros = fake
            d_fake_error.backward()
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()
            d_error = d_real_error + d_fake_error
            D_losses.append(d_error.detach().numpy())
#             print(f'd:{d_error}', end=' ')
        for g_index in range(g_steps):
        # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()

            gen_input = Variable(torch.rand(BATCH_SIZE, g_input_size))
            g_fake_data = G(gen_input)
            dg_fake_decision = D(g_fake_data)
            g_error = criterion(dg_fake_decision, Variable(torch.ones(BATCH_SIZE, 1)))  # we want to fool, so pretend it's all genuine

            g_error.backward()
            g_optimizer.step()  # Only optimizes G's parameters
            
            G_losses.append(g_error.detach().numpy())
#             print(f'g:{g_error}')
    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
        epoch , EPOCH, np.mean(D_losses), np.mean(G_losses)))

In [10]:
def test(epoch):
    G.eval()
    

In [ ]:
des_path = project_root + 'results/'
if not os.path.exists(des_path):
    os.makedirs(des_path, exist_ok=True)

for epoch in range(1, EPOCH + 1):
    train(epoch)
#     test(epoch)
    with torch.no_grad():
        sample = torch.randn(BATCH_SIZE, g_input_size).to(device)
        sample = G(sample).cpu()
        save_image(sample.view(BATCH_SIZE, 1, 28, 28),
                   f'{des_path}epoch_{epoch}.png')


[1/5000]: loss_d: 1.258, loss_g: 1.327
[2/5000]: loss_d: 1.338, loss_g: 0.975
[3/5000]: loss_d: 1.381, loss_g: 0.923
[4/5000]: loss_d: 1.394, loss_g: 0.813
[5/5000]: loss_d: 1.385, loss_g: 0.812
[6/5000]: loss_d: 1.402, loss_g: 0.842
[7/5000]: loss_d: 1.396, loss_g: 0.793
[8/5000]: loss_d: 1.403, loss_g: 0.765
[9/5000]: loss_d: 1.389, loss_g: 0.802
[10/5000]: loss_d: 1.396, loss_g: 0.795
[11/5000]: loss_d: 1.386, loss_g: 0.913
[12/5000]: loss_d: 1.379, loss_g: 0.849
[13/5000]: loss_d: 1.388, loss_g: 0.748
[14/5000]: loss_d: 1.402, loss_g: 0.834
[15/5000]: loss_d: 1.393, loss_g: 0.779
[16/5000]: loss_d: 1.402, loss_g: 0.788
[17/5000]: loss_d: 1.403, loss_g: 0.773
[18/5000]: loss_d: 1.397, loss_g: 0.786
[19/5000]: loss_d: 1.390, loss_g: 0.770
[20/5000]: loss_d: 1.394, loss_g: 0.776
[21/5000]: loss_d: 1.399, loss_g: 0.867
[22/5000]: loss_d: 1.399, loss_g: 0.794
[23/5000]: loss_d: 1.389, loss_g: 0.774
[24/5000]: loss_d: 1.386, loss_g: 0.819
[25/5000]: loss_d: 1.375, loss_g: 0.768
[26/5000]

训练极不稳定，随着训练过程发展会出现：
- 判别器损失趋近于0， 生成器损失增大，说明学得了很好的判别器，而生成器就被压制了

现在训练数据位于[0,1)之间，规范化到[-1,1]之间效果会怎样？